# 神经网络

将对于数据的操作的各个模块或层次组织在一起，就是神经网络模型。`torch.nn`模块提供了所有的用于建立神经网络模型的模块的命名空间(隐藏层、输出层等)。所有的模型都是`nn.Moudle`的子类。一个神经网络模型可以包括许多的其他的模型，这样就可以建立很复杂的结构。

In [1]:
# 导入相应的包
import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 获取可以执行的设备
print("计算运行设备:", device)

计算运行设备: cuda


# 建立模型
可以通过继承`nn.Module`类来实现模型的建立，然后在`__init__`函数里初始化建立模型。所有的Module子类都需要实现前向传播函数`forward`,来实现对输入数据的处理。

In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()   # 将图像扁平化
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )

    def forward(self, x):
        x = self.flatten(x)  # 获取每一batch的输入，并利用第一个模块扁平化
        logits = self.linear_relu_stack(x) # 放入第二个复杂的模型进行训练
        return logits
    
# 将模型放到对应的结构上
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# 使用模型
为了使用模型，可以直接向模型传递输入数据，上面的模型最后将会输出一个10维的`tensor`结构，这些操作都是在后台进行的，无需显式的调用`forward`函数进行计算。因为这是一个多分类问题，将上述的10维的结构放入一个`softmax`分类器里面，就可以得到概率最大的那个类别，也就是最终的分类。

In [3]:
x = torch.rand(1, 28, 28, device=device)
print("x",x)
logits = model(x)
print("logits",logits)
pred_probab = nn.Softmax(dim=1)(logits)
print("pred_probab",pred_probab)
y_pred = pred_probab.argmax(1)
print("y_pred",y_pred)

x tensor([[[7.8331e-01, 4.0430e-01, 1.5309e-01, 9.0927e-01, 6.9913e-01,
          4.6765e-01, 9.0149e-01, 7.4656e-01, 4.8227e-01, 2.0388e-01,
          8.5696e-01, 1.5308e-01, 2.7565e-01, 2.7932e-01, 7.2258e-01,
          1.5606e-01, 4.0048e-01, 6.8498e-01, 3.0512e-01, 5.2559e-01,
          6.0010e-01, 5.2044e-01, 8.6385e-01, 6.7460e-01, 1.9618e-01,
          7.0253e-01, 3.0140e-01, 3.8909e-01],
         [6.2238e-01, 2.8826e-01, 7.0873e-01, 4.3049e-01, 6.6013e-01,
          6.5122e-02, 8.0897e-01, 5.9807e-01, 9.5499e-01, 3.7352e-02,
          3.6375e-01, 9.9677e-01, 4.2074e-01, 7.6707e-01, 6.4430e-02,
          6.0024e-01, 6.3686e-01, 2.7674e-01, 4.5303e-01, 7.5484e-01,
          7.7232e-01, 9.8329e-01, 1.0953e-01, 3.7313e-01, 3.8552e-01,
          4.4290e-01, 8.3544e-01, 5.6694e-01],
         [3.3093e-01, 9.1029e-01, 9.3298e-01, 3.1736e-01, 3.8311e-01,
          5.5716e-01, 9.6307e-01, 3.4901e-01, 1.6077e-01, 5.8234e-01,
          4.6331e-01, 9.1416e-01, 8.7926e-01, 9.0364e-01, 6.5101

由上面可以看出来预测的变化，经过分类器之后，可以得到概率最大的那个类别，也就是随即的类别。**但是这里其实并没有真正的训练，只是将随即的变量代入了模型，模型内部的所有的参数都是初始化来的，并不是学习来的。**，此处只是简单的建立了模型，模型的训练在后面。

# 模型层次详解

## nn.Flatten层
顾名思义，这是一个扁平化层，是将输入的图像，本来是三个通道每个通道的有`28*28`个像素的图片，现在将这个28*28个像素展开，即每个通道都展开，这样做的好处就是一次性可以将所有的像素输入进去。

## nn.Linear 层
这是一个将线性转换器应用到模型的一层，利用其自身存储好的权重(weight)和偏移值(bias),来
进行线性计算。

## nn.ReLU层
这是非线性的激活函数，用于创建复杂的输入与输出之间的映射。放在了线性模型的输入之后，目的是引入非线性，帮助神经网络模型来学习更加丰富多彩的现象。

## nn.Sequential 层
这是一个有序的模型容器。输入数据被放入这个序列中，依次按照序列的定义来对数据进行操作。可以使用Sequential容器来快捷的创立一个模型。

## nn.Softmax 层
模型的上一层输出的是一个logits tensor，是一个介于无穷小到无穷大之间的数字。这个数字我们没有办法拿来预测，因此需要将这些值按比例缩小到[0,1]之间，并且每一个类别的概率之和最终等于一，便可以利用概率的知识，来判断预测的类别。其中的`dim=1`表示的就是最终的值的和必须符合的维度。

In [6]:
# 模型层次详解测试
input_data = torch.rand(3,28,28)
print(input_data.size())

# 展开层
flatten = nn.Flatten()
flat_image = flatten(input_data)
print(flat_image.size())

# 线性层
linear = nn.Linear(28*28,20)
hidden = linear(flat_image)
print(hidden.size())

# ReLU层
hidden = nn.ReLU()(hidden)
print(hidden.size())

# Sequential 容器
seq_modules = nn.Sequential(
    flatten,
    linear,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

## softmax 层
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

torch.Size([3, 28, 28])
torch.Size([3, 784])
torch.Size([3, 20])
torch.Size([3, 20])
tensor([[ 0.1568, -0.0936, -0.0447,  0.3378,  0.0892, -0.0914,  0.3255, -0.1408,
          0.3747,  0.1576],
        [ 0.1487,  0.0172, -0.0119,  0.2713,  0.0475, -0.0244,  0.3693, -0.1314,
          0.3427,  0.1524],
        [ 0.1345, -0.0866,  0.0135,  0.3104,  0.0996, -0.1466,  0.2608, -0.1119,
          0.4364,  0.0819]], grad_fn=<AddmmBackward>)
tensor([[0.1033, 0.0804, 0.0845, 0.1238, 0.0966, 0.0806, 0.1223, 0.0767, 0.1285,
         0.1034],
        [0.1018, 0.0892, 0.0867, 0.1151, 0.0920, 0.0856, 0.1269, 0.0769, 0.1236,
         0.1022],
        [0.1019, 0.0816, 0.0902, 0.1214, 0.0984, 0.0769, 0.1156, 0.0796, 0.1378,
         0.0966]], grad_fn=<SoftmaxBackward>)


# 模型参数
在模型的显式训练的过程中，是有很多的参数的。我们可以将这些参数打印出来，nn.Module会自动跟踪定义的所有的模型，并保存其参数值。可以使用模型的`parameters()`或者`named_parameters()`方法来获取模型的参数。

In [7]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0292,  0.0343,  0.0295,  ..., -0.0068, -0.0105,  0.0169],
        [-0.0252,  0.0179,  0.0225,  ...,  0.0110, -0.0297,  0.0067]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0129, 0.0053], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0431, -0.0155,  0.0387,  ..., -0.0185, -0.0389, -0.0313],
        [ 0.0140, -0.0137,  0.0157,  ...,  0.0333, -0.0100,  0.0283]],
       device='cuda:0', grad_fn=<Slic

over